In [ ]:
# !pip install torchange

In [ ]:
import torchange
import geoai
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from geoai.change_detection import ChangeDetection
from IPython.display import Image

In [ ]:
from libs.utils import get_device, visualize_comparison
device = get_device()

from libs.ee import initialize_ee
ee = initialize_ee(get_ipython())

In [ ]:
pre_dir_marks = "./data/4_change_detection/marks"
post_dir_marks = "./data/2-data-explore-events/marks"

# get list of images in pre_dir
pre_img_marks = [file for file in os.listdir(pre_dir_marks)]
post_img_marks = [file for file in os.listdir(post_dir_marks)]

pre_img_marks.sort()
post_img_marks.sort()

for idx, img_name in enumerate(pre_img_marks[:10]):
    pre_image_path = f"{pre_dir_marks}/{img_name}"
    post_image_path = f"{post_dir_marks}/{post_img_marks[idx]}"

    visualize_comparison(pre_image_path, post_image_path, left_title=img_name, right_title=post_img_marks[idx])

In [ ]:
pre_dir = "./data/4_change_detection/rgb"
post_dir = "./data/2-data-explore-events/rgb"

# get list of images in pre_dir
pre_imgs = [file for file in os.listdir(pre_dir)]
post_imgs = [file for file in os.listdir(post_dir)]

In [ ]:
idx = 0

pre_dir_rgb = "./data/4_change_detection/rgb"
post_dir_rgb = "./data/2-data-explore-events/rgb"

pre_rgb_files = sorted([f for f in os.listdir(pre_dir_rgb) if f.endswith('.png')])
post_rgb_files = sorted([f for f in os.listdir(post_dir_rgb) if f.endswith('.png')])

test_pre_img = f"{pre_dir_rgb}/{pre_rgb_files[idx]}"
test_post_img = f"{post_dir_rgb}/{post_rgb_files[idx]}"

pre_rgb_files[idx], post_rgb_files[idx]

In [ ]:
out_folder = "./data/5-change_detection/detection_results"
Path(out_folder).mkdir(exist_ok=True, parents=True)

print(f"Working directory: {out_folder}")

In [ ]:
# Make sure model directory exists
# to store downloaded model weights
Path("~/.cache/torch/hub/checkpoints/").expanduser().mkdir(parents=True, exist_ok=True)

# Initialize change detection
detector = ChangeDetection(sam_model_type="vit_h")

# Configure parameters (following the torchange example)
detector.set_hyperparameters(
    change_confidence_threshold=145,
    use_normalized_feature=True,
    bitemporal_match=True,
)

detector.set_mask_generator_params(
    points_per_side=32,
    stability_score_thresh=0.95,
)

print("Change detection system initialized!")

In [ ]:
# Check AnyChange class signature
from torchange.models.segment_any_change import AnyChange
import inspect

print("AnyChange signature:")
print(inspect.signature(AnyChange.__init__))

In [ ]:
# Since the images are PNG without geospatial metadata, we need to convert them to GeoTIFF
# with dummy geospatial information so the geoai library can process them

import rasterio
from rasterio.transform import from_bounds
from PIL import Image as PILImage
import numpy as np

# Load PNG images
img1 = np.array(PILImage.open(test_pre_img))
img2 = np.array(PILImage.open(test_post_img))

print(f"Image 1 shape: {img1.shape}")
print(f"Image 2 shape: {img2.shape}")

# Create temporary GeoTIFF files with dummy geospatial information
temp_pre_tif = f"{out_folder}/temp_pre.tif"
temp_post_tif = f"{out_folder}/temp_post.tif"

# Define dummy bounds (same for both images so they overlap perfectly)
height, width = img1.shape[:2]
bounds = (0, 0, width, height)  # left, bottom, right, top
transform = from_bounds(bounds[0], bounds[1], bounds[2], bounds[3], width, height)

# Write pre-event image as GeoTIFF
with rasterio.open(
    temp_pre_tif, 'w',
    driver='GTiff',
    height=height,
    width=width,
    count=3,
    dtype=img1.dtype,
    crs='EPSG:4326',  # Dummy CRS
    transform=transform
) as dst:
    for i in range(3):
        dst.write(img1[:, :, i], i + 1)

# Write post-event image as GeoTIFF
with rasterio.open(
    temp_post_tif, 'w',
    driver='GTiff',
    height=height,
    width=width,
    count=3,
    dtype=img2.dtype,
    crs='EPSG:4326',  # Dummy CRS
    transform=transform
) as dst:
    for i in range(3):
        dst.write(img2[:, :, i], i + 1)

print(f"\nTemporary GeoTIFF files created")

# Now run change detection with the GeoTIFF files
results = detector.detect_changes(
    temp_pre_tif,
    temp_post_tif,
    output_path=f"{out_folder}/binary_mask.tif",
    export_probability=True,
    probability_output_path=f"{out_folder}/probability_mask.tif",
    export_instance_masks=True,
    instance_masks_output_path=f"{out_folder}/instance_masks.tif",
    return_detailed_results=True,
    return_results=False,
)

print(f"\nChange detection completed!")
print(f"Total instances detected: {results['summary']['total_masks']}")
print(f"Image size: {results['summary']['original_shape']}")

In [ ]:
# probability visualization
detector.visualize_results(
    test_pre_img,
    test_post_img,
    f"{out_folder}/binary_mask.tif",
    f"{out_folder}/probability_mask.tif",
)